In [6]:
# Load data
with open("../data/day19test.txt", "r") as f:
    input = f.read()

parts = input.split("\n\n")
rules = parts[0]
msgs = parts[1]

# real data is not sorted!
rule_list = [rule[3:] for rule in rules.split("\n")]
rule_list

['4 1 5', '2 3 | 3 2', '4 4 | 5 5', '4 5 | 5 4', '"a"', '"b"']

In [88]:
# Puzzle 1
# pyparsing?
# build list of allowed options? might be too much
import re

allowed_options = []

regex_rule = re.compile(r"(\d+ \d+ \| \d+ \d+)|((?:\d+ ?)+)|\"([ab])\"")

k = 0
i = 0
while k < 6: # < make recursive function
    # do regex and named groups for main rules
    results = regex_rule.search(rule_list[k])
    groups = results.groups()
    if groups[0]: # 2 3 | 3 2
        # do with regex
        print(groups[0])
    if groups[1]: # 4 1 5
        print(groups[1])
        for rule_n in groups[1].split(" "):
            print(rule_n)
            # call some function, recursion
    if groups[2]: # a b
        print(groups[2])
        if groups[2] == "a":
            allowed_options.append("a")
        if groups[2] == "b":
            allowed_options.append("b")
    k += 1
allowed_options


4 1 5
4
1
5
2 3 | 3 2
4 4 | 5 5
4 5 | 5 4
a
b


['a', 'b']